In [103]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pyarrow.parquet as pq 
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

# Data Info

In [ ]:
# Load the metadata of the parquet files
metadata_a = pq.read_metadata('A/X_train_observed.parquet')
metadata_b = pq.read_metadata('B/X_train_observed.parquet')
metadata_c = pq.read_metadata('C/X_train_observed.parquet')

# Get the schema of the parquet files
schema_a = metadata_a.schema
schema_b = metadata_b.schema
schema_c = metadata_c.schema

print("Schema for file A:")
print(schema_a)
print("\nSchema for file B:")
print(schema_b)
print("\nSchema for file C:")
print(schema_c)

# DATA LOADING

In [95]:
# Load the data
df_A = pq.read_table('A/X_train_observed.parquet').to_pandas()
df_B = pq.read_table('B/X_train_observed.parquet').to_pandas()
df_C = pq.read_table('C/X_train_observed.parquet').to_pandas()
df_A_targets = pq.read_table('A/train_targets.parquet').to_pandas()
df_A_targets = df_A_targets.rename(columns={'time': 'date_forecast'})
df_B_targets = pq.read_table('B/train_targets.parquet').to_pandas()
df_B_targets = df_B_targets.rename(columns={'time': 'date_forecast'})
df_C_targets = pq.read_table('C/train_targets.parquet').to_pandas()
df_C_targets = df_C_targets.rename(columns={'time': 'date_forecast'})

# DATA PREPROCESSING

In [194]:
# TODO:
# - Create method for joining target dataset with features dataset 
# - Create method for creating horizon dataframes
# - identify which features to use for lag features 

# Class for general feature processing
class FeatureProcessingClass():
    def __init__(self):

        ###--- DATA SPECIFIC CLASS VARIABLES---###

        # all features 
        self.features = ['date_forecast', 
                          'absolute_humidity_2m:gm3', 
                          'air_density_2m:kgm3', 
                          'ceiling_height_agl:m', 
                          'clear_sky_energy_1h:J', 
                          'clear_sky_rad:W', 
                          'cloud_base_agl:m', 
                          'dew_or_rime:idx', 
                          'dew_point_2m:K', 
                          'diffuse_rad:W', 
                          'diffuse_rad_1h:J', 
                          'direct_rad:W', 
                          'direct_rad_1h:J', 
                          'effective_cloud_cover:p', 
                          'elevation:m', 
                          'fresh_snow_12h:cm', 
                          'fresh_snow_1h:cm', 
                          'fresh_snow_24h:cm', 
                          'fresh_snow_3h:cm', 
                          'fresh_snow_6h:cm', 
                          'is_day:idx', 
                          'is_in_shadow:idx', 
                          'msl_pressure:hPa', 
                          'precip_5min:mm', 
                          'precip_type_5min:idx', 
                          'pressure_100m:hPa', 
                          'pressure_50m:hPa', 
                          'prob_rime:p', 
                          'rain_water:kgm2', 
                          'relative_humidity_1000hPa:p', 
                          'sfc_pressure:hPa', 
                          'snow_density:kgm3', 
                          'snow_depth:cm', 
                          'snow_drift:idx', 
                          'snow_melt_10min:mm', 
                          'snow_water:kgm2', 
                          'sun_azimuth:d', 
                          'sun_elevation:d', 
                          'super_cooled_liquid_water:kgm2', 
                          't_1000hPa:K', 
                          'total_cloud_cover:p', 
                          'visibility:m', 
                          'wind_speed_10m:ms', 
                          'wind_speed_u_10m:ms', 
                          'wind_speed_v_10m:ms', 
                          'wind_speed_w_1000hPa:ms']

        # Categorical columns (specify for XGBoost)
        self.categorical_features = ['dew_or_rime:idx',
                                    'is_day:idx', 
                                    'is_in_shadow:idx', 
                                    'precip_type_5min:idx', 
                                    'snow_drift:idx', 
                                    'is_weekend']
        
        # fetaures not suited for lag features 
        self.non_lag_features = ['date_forecast',  'ceiling_height_agl:m', 'elevation:m'] + self.categorical_features

        # features to be replicated with lagged values
        self.lag_features = [feature for feature in self.features if feature not in self.non_lag_features]
        self.lag_values = [1, 12, 24]
        
        # join features
        self.date_forecast = 'date_forecast'
        
        # time features
        self.time_features = ['year', 'quarter', 'month', 'week', 'hour', 'day_of_year', 'day', 'weekday', 'is_weekend']
        
        # target features
        self.target_features = 'pv_measurement'
        
        # Columns of latitude & longitude
        self.lat_lon_columns = ['latitude', 'longitude'] # will not be used for now


    ###--- METHODS FOR JOINING DATASETS ---###


    # method for modifying column names that datasets to be joined have in common
    def add_suffix_to_column_names(self, df: pd.DataFrame, suffix: str, columns_no_change: list[str]): # Needed when we combine with other datasets than the target dataset
        '''
        Change column names by given suffix, keep columns_no_change, and return back the data

        PARAMS:
        - df: data
        - suffix: suffixes to add to column names
        - columns_no_change: list of column names who should not be changed
        '''
        df.columns = [col + suffix 
                      if col not in columns_no_change
                      else col
                      for col in df.columns]
        return df
    

    # generalized method for cropping dataset rows so one or more datasets to be joined match in time
    def crop_datasets(self, df_list: list[pd.DataFrame]): # mabye not needed since we are using inner join in join_datasets function
        '''
        Crop datasets so they match in time

        PARAMS:
        - df_list: list of dataframes to be cropped
        RETURNS:
        - df_cropped: list of cropped dataframes with matching time intervals
        '''
        # Find the first and last date in all datasets 
        min_dates = [df['date_forecast'].min() for df in df_list] 
        max_dates = [df['date_forecast'].max() for df in df_list] 
        interval = [max(min_dates), min(max_dates)]
        # Crop the datasets
        df_cropped = [df[(df['date_forecast'] >= interval[0]) & (df['date_forecast'] <= interval[1])] for df in df_list]
        return df_cropped
        
    
    # method for joining target dataset with features dataset
    def join_datasets(self, df_list: list[pd.DataFrame], on='date_forecast'):
        '''
        Join dataset in list on 'date_forecast' column

        PARAMS:
        - df: data
        - target_df: target data
        RETURNS:
        - df: joined data
        '''
        # Check if all datasets have the same 'date_forecast' column
        if not all([on in df.columns for df in df_list]):
            raise ValueError('Not all datasets have'+ on +'column') # changed from f-string to string
        
        # Join datasets
        df = df_list[0]
        for i in range(1, len(df_list)):
            df = df.merge(df_list[i], on=on, how='left') # changed from inner to left join
        return df
    

    ###--- METHODS FOR MODIFYING FEATURES ---###


    def impute_missing_values(self, df: pd.DataFrame, time_column='date_forecast'):
        '''
        Impute missing values with mean of the two closest non-NaN values

        PARAMS:
        - df: pandas data
        '''
        # Create a KNNImputer
        imputer = KNNImputer(n_neighbors=2)

        # Save the 'date_forecast' column and remove it from the DataFrame
        date_forecast = df[time_column]
        df = df.drop(columns=[time_column])

        # Fit and transform the DataFrame
        df_imputed = imputer.fit_transform(df)
        df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

        # Add the 'date_forecast' column back to the DataFrame
        df_imputed[time_column] = date_forecast

        return df_imputed
    

    ###--- METHODS FOR CREATING NEW FEATURES ---###

    
    def create_time_features(self, df: pd.DataFrame):
        '''
        Create data features based on datetime column

        PARAMS:
        - df: data
        RETURNS:
        - df: data with time features
        '''

        # Check if 'date_forecast' column exists
        if 'date_forecast' not in df.columns.tolist():
            raise ValueError("DataFrame does not have 'date_forecast' column")

        # Try to convert 'date_forecast' to datetime
        try:
            df['date_forecast'] = pd.to_datetime(df['date_forecast'])
        except Exception as e:
            raise ValueError("Cannot convert 'date_forecast' to datetime: " + str(e))

        # time period features
        df['year'] = df['date_forecast'].dt.year
        df['quarter'] = df['date_forecast'].dt.quarter
        df['month'] = df['date_forecast'].dt.month
        df['week'] = df['date_forecast'].dt.isocalendar().week
        df['hour'] = df['date_forecast'].dt.hour

        # day features
        df['day_of_year'] = df['date_forecast'].dt.day_of_year # mabye not needed
        df['day'] = df['date_forecast'].dt.day # mabye not needed
        df['weekday'] = df['date_forecast'].dt.weekday

        # boolean features
        df['is_weekend'] = df['weekday'] > 5

        return df
    

    def create_lag_features(self, df: pd.DataFrame, lag_features: list[str], lag_values: list[int]):
        '''
        Create lag features for given columns

        PARAMS:
        - df: data
        - lag_features: list of columns to create lag features for
        - lag_values: list of lag values
        RETURNS:
        - df: data with lagged features
        '''
        # checking that not any lag features is categorical
        if any(feature in lag_features for feature in self.categorical_features):
            catagorical_feature_list = [feature for feature in lag_features if feature in self.categorical_features]
            raise ValueError(f"Cannot create lag features for categorical features {catagorical_feature_list}")

        for feature in lag_features:
            for lag in lag_values:
                df[feature + '_lag_' + str(lag)] = df[feature].shift(lag)
        return df
    

    ###--- METHODS FOR DATA CONVERSION/MANIPULATION ---###


    def convert_to_categorical(self, df: pd.DataFrame, categorical_features: list[str]):
        '''
        Convert columns to categorical dtype

        PARAMS:
        - df: data
        - catagorical_features: list of columns to convert to categorical
        '''
        return  df[categorical_features].apply(lambda x: x.astype('category'))
    

    def convert_to_numerical(self, df: pd.DataFrame, numerical_features: list[str]):
        '''
        Convert columns to numerical dtype
        
        PARAMS:
        - df: data
        - numerical_features: list of columns to convert to numerical
        '''
        return  df[numerical_features].apply(lambda x: x.astype('float'))
    

    ###--- METHODS FOR CREATING SHIFTED DATAFRAME FOR EACH FORECASTED HOUR ---###


    def create_horizon_dataframes(self, df: pd.DataFrame, horizon = 24):
        '''
        Create a shifted dataframe for each forecasted hour

        PARAMS:
        - df: data
        - horizon: number of timesteps forecasted
        RETURNS:
        - dataframes: dictionary of dataframes for each forecasted hour
        '''
        # dictionary to store dataframes - Key: forecasted hour, Value: shifted dataframe
        dataframes = {}
        for h in range(1, horizon + 1):
            df_h = df.copy()
            df_h[f'{self.target_features}_horizon_{h}'] = df_h[self.target_features].shift(-h)
            for feature in self.time_features:
                df_h[feature] = df_h[feature].shift(-h)
            df_h = df_h.dropna()
            dataframes[h] = df_h.drop(columns=[self.target_features])
    
        return dataframes
        

    ###--- CALL METHOD FOR INITIALISING CLASS AS FUNCTION ---###


    def __call__(self, data: list[pd.DataFrame]):
        '''
        Processing of features from all datasets 
        PARAMS: 
        - data: list of processed dataframes containg all datasets that are to be joined and preprocessed
        RETURNS:
        - df: List of 24 dataframes with features for each forecasted hour
        '''
        # joining datasets
        data = self.join_datasets(data)
        # Impute missing values
        data = self.impute_missing_values(data)
        # Create features
        data = self.create_lag_features(data, self.lag_features, self.lag_values)
        data = self.create_time_features(data)
        # Change columns to categorical for XGBoost
        data = data.drop(columns=['date_forecast'])
        data[self.categorical_features] = data[self.categorical_features].astype('category')
        # Create horizon dataframes
        data = self.create_horizon_dataframes(data)
        return data

In [129]:
# Function to split a single dataframe into train and test sets based on chronological order
def split_dataframe(df, train_size=0.8):
    split_index = int(len(df) * train_size)
    train_df = df.iloc[:split_index]
    test_df = df.iloc[split_index:]
    return train_df, test_df


# Function to split a list of dataframes into train, test  and validation dicts
def train_test_val_split(df_dict, train_size=0.8, val_size=0.1):
    train_dict = {}
    test_dict = {}
    val_dict = {}
    for df in df_dict.items():
        train_df, test_df = split_dataframe(df[1], train_size) # test set is 0.2 0f total set
        test_df, val_df = split_dataframe(test_df, val_size) # validation set is 0.1 of test set
        train_dict[df[0]] = train_df
        test_dict[df[0]] = test_df
        val_dict[df[0]] = val_df
    return train_dict, test_dict, val_dict

# MODEL IMPLEMENTATION

In [179]:
class xgboost_forecasting():
    def __init__(self, train_data, test_data, val_data):
        
        # Global dict variable to store 
        # the best model: 24 models, 
        # errors: mse for each induvidual horizon model, 
        # predictions from test set, 
        # r2 score for each induvidual horizon model 
        self.best_model = None
        
        # Global variable to store the data
        self.train_data = train_data
        self.test_data = test_data
        self.val_data = val_data

    # Function to be minimized by optuna
    def objective(self, trial):
        '''
        Uses optuna to optimize hyperparameters for XGBoost and return best model
        PARAMS:
        - trial: optuna trial object
        RETURNS:
        - rmse: root mean squared error of the model
        '''

        # Define hyperparameters and their ranges
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 10.0),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500)
        }

        # Train the model
        models = self.fitmod(params)

        # Predict and evaluate
        pred, err, r2 = self.predmod(models)
        rmse = np.sum(list(err.values()))
        
        # Update the best model
        if self.best_model is None or rmse < self.best_model['rmse']:
            self.best_model = {'models': models, 'rmse': rmse, 'preds': pred, 'errors': err, 'r2': r2}

        return rmse


    def create_study(self):
        ''' 
        RETURNS:
        - study: optuna study object with best hyperparameters 
        '''


        study = optuna.create_study(direction='minimize')
        study.optimize(self.objective, n_trials=100)

        # Print the best parameters
        print('Best trial:')
        trial = study.best_trial

        print(f'  Value: {trial.value}')
        print('  Params: ')
        for key, value in trial.params.items():
            print(f'    {key}: {value}') 
    

    def fitmod(self, params):
        # Define and train models for each forecasted hour
        models = {}
        reg = xgb.XGBRegressor(**params, enable_categorical=True, objective='reg:absoluteerror', n_jobs=-1)
        for i in range(1, 25):
            # define training sets
            X_train = self.train_data[i].drop(columns=['pv_measurement_horizon_' + str(i)])
            y_train = self.train_data[i]['pv_measurement_horizon_' + str(i)]
            # define validation sets
            X_val = self.val_data[i].drop(columns=['pv_measurement_horizon_' + str(i)])
            y_val = self.val_data[i]['pv_measurement_horizon_' + str(i)]
            # fit model
            reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
            models[i] = reg
        return models
    

    def predmod(self, models):
        '''Make one prediction per forecasted hour'''
        preds = {}
        errors = {}
        r2 = {}
        for i in range(1, 25):
            X_test = self.test_data[i].drop(columns=['pv_measurement_horizon_' + str(i)])
            y_test = self.test_data[i]['pv_measurement_horizon_' + str(i)]
            # making predictions
            preds[i] = models[i].predict(X_test)
            # calculating errors (MSE)
            errors[i] = mean_squared_error(y_test, preds[i], squared=False)
            # calculating R^2 
            r2[i] = r2_score(y_test, preds[i])
        return preds, errors, r2


    def __call__(self, fit = True):
        if fit:
            # fit model 
            study = self.create_study()
        else:
            print('model values: ') 
            for key, value in self.best_model.items():
                print(f'    {key}: {value}')
        return study

# MODEL FITTING

In [195]:
FeatureProcessor = FeatureProcessingClass()

data = FeatureProcessor([df_B, df_B_targets])

/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/3514504953.py:235: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feature + '_lag_' + str(lag)] = df[feature].shift(lag)
/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/3514504953.py:235: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feature + '_lag_' + str(lag)] = df[feature].shift(lag)
/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/3514504953.py:235: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [196]:
tr, te, va = train_test_val_split(data)

In [197]:
xgb_object = xgboost_forecasting(tr, te, va)

In [198]:
study = xgb_object(fit = True)

[I 2024-06-24 15:55:28,260] A new study created in memory with name: no-name-55c5d106-e2a8-4b9c-bf25-40b8f316da77
/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/475161506.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/475161506.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 10.0),
/var/folders/n7/1xc9jn1s31b382qxd82wbp5r0000gn/T/ipykernel_7368/475161506.py:31: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will 

KeyboardInterrupt: 